In [1]:
from catboost import Pool, CatBoostClassifier

import csv

from datetime import datetime

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="ask")
from geopy.distance import distance
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0.01, max_retries = 10)

from lightgbm import LGBMClassifier

import numpy as np 
import matplotlib.pyplot as plt
import ml_metrics as metrics
import pandas as pd

import pycountry

from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingRegressor, StackingClassifier, \
GradientBoostingClassifier, VotingClassifier, BaggingClassifier
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, mean_squared_error, average_precision_score, accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split, KFold, GridSearchCV, StratifiedShuffleSplit
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from tqdm import tqdm

from xgboost import XGBClassifier

import os
for dirname, _, filenames in os.walk('Data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

Data/d.csv
Data/lat_long.csv
Data/tours_useful.csv
Data/bikers_useful.csv
Data/bikers_network_useful.csv
Data/test_tids.csv
Data/sample_submission.csv
Data/x_p.csv
Data/test_x.csv
Data/d_p.csv
Data/test_bikerids.csv
Data/x_1.csv
Data/tour_convoy.csv
Data/test.csv
Data/probabs.csv
Data/locations.csv
Data/evalml_debug.log
Data/a.csv
Data/test_x_2.csv
Data/x_2.csv
Data/xy_1.csv
Data/x_n.csv
Data/bikers.csv
Data/y_1.csv
Data/y_2.csv
Data/y.csv
Data/tours.csv
Data/x_3.csv
Data/bikers_network.csv
Data/useful.tar.xz
Data/x.csv
Data/train.csv
Data/tour_convoy_useful.csv
Data/imputed/tours_useful.csv
Data/imputed/bikers_useful.csv
Data/non_imputed/tours_useful.csv
Data/non_imputed/bikers_useful.csv
Data/khushal/training_data_final_1.csv
Data/khushal/testing_data_final_1.csv


In [2]:
train = pd.read_csv("Data/train.csv")
test = pd.read_csv("Data/test.csv")
biker = { key: row for key, *row in csv.reader(open("Data/bikers_useful.csv", 'r'))}
print(biker.pop('biker_id'))
biker_net = { key: row for key, *row in csv.reader(open("Data/bikers_network_useful.csv", 'r'))}
print(biker_net.pop('biker_id'))
tours = { key: row for key, *row in csv.reader(open("Data/tours_useful.csv", 'r'))}
print(tours.pop('tour_id'))
tour_convoy = { key: row for key, *row in csv.reader(open("Data/tour_convoy_useful.csv", 'r'))}
print(tour_convoy.pop('tour_id'))

['language_id', 'location_id', 'bornIn', 'gender', 'member_since', 'area', 'time_zone', 'latitude', 'longitude']
['friends']
['biker_id', 'tour_date', 'city', 'state', 'pincode', 'country', 'latitude', 'longitude', 'w1', 'w2', 'w3', 'w4', 'w5', 'w6', 'w7', 'w8', 'w9', 'w10', 'w11', 'w12', 'w13', 'w14', 'w15', 'w16', 'w17', 'w18', 'w19', 'w20', 'w21', 'w22', 'w23', 'w24', 'w25', 'w26', 'w27', 'w28', 'w29', 'w30', 'w31', 'w32', 'w33', 'w34', 'w35', 'w36', 'w37', 'w38', 'w39', 'w40', 'w41', 'w42', 'w43', 'w44', 'w45', 'w46', 'w47', 'w48', 'w49', 'w50', 'w51', 'w52', 'w53', 'w54', 'w55', 'w56', 'w57', 'w58', 'w59', 'w60', 'w61', 'w62', 'w63', 'w64', 'w65', 'w66', 'w67', 'w68', 'w69', 'w70', 'w71', 'w72', 'w73', 'w74', 'w75', 'w76', 'w77', 'w78', 'w79', 'w80', 'w81', 'w82', 'w83', 'w84', 'w85', 'w86', 'w87', 'w88', 'w89', 'w90', 'w91', 'w92', 'w93', 'w94', 'w95', 'w96', 'w97', 'w98', 'w99', 'w100', 'w_other']
['going', 'maybe', 'invited', 'not_going']


In [3]:
train_bikers_set = np.array(train["biker_id"].drop_duplicates())
test_bikers_set = np.array(test["biker_id"].drop_duplicates())
req_bikers_set = np.array([*biker])
tours_set = np.array([*tours])
req_bikers_set.shape, train_bikers_set.shape, tours_set.shape

((2231,), (1934,), (9343,))

In [4]:
tour_convoy_full = { key: row for key, *row in csv.reader(open("Data/tour_convoy.csv", 'r'))}

In [5]:
events_attended = {}
events_maybe = {}
events_invited = {}
events_notgoing = {}
for tid, row in tour_convoy_full.items():
    people_attended = row[0].split()
    people_maybe = row[1].split()
    people_invited = row[2].split()
    people_notgoing = row[3].split()
    for bid in people_attended:
        events_attended[bid] = events_attended.get(bid, 0) + 1
    for bid in people_maybe:
        events_maybe[bid] = events_maybe.get(bid, 0) + 1
    for bid in people_invited:
        events_invited[bid] = events_invited.get(bid, 0) + 1
    for bid in people_notgoing:
        events_notgoing[bid] = events_notgoing.get(bid, 0) + 1
        
events_attended_friends = {}
events_maybe_friends = {}
events_invited_friends = {}
events_notgoing_friends = {}
no_friends = {}
for bid, row in biker_net.items():
    friends = row[0].split()
    no_friends[bid] = len(friends)
    for friend in friends:
        events_attended_friends[bid] = events_attended_friends.get(bid, 0) + events_attended.get(friend, 0)
        events_maybe_friends[bid] = events_maybe_friends.get(bid, 0) + events_maybe.get(friend, 0)
        events_invited_friends[bid] = events_invited_friends.get(bid, 0) + events_invited.get(friend, 0)
        events_notgoing_friends[bid] = events_notgoing_friends.get(bid, 0) + events_notgoing.get(friend, 0)
        
events_attended_friends

{'IF1286067': 10,
 'EB01519751': 54,
 'CB63304477': 25,
 'DJ7493741': 14,
 'DI3523902': 75,
 'DF54343061': 8,
 'BF38685648': 150,
 'DC51568850': 32,
 'CI79905973': 52,
 'DA95718598': 8,
 'CA08761267': 253,
 'EI3332100': 38,
 'DA71657222': 36,
 'BJ48333652': 21,
 'BI72064755': 0,
 'DB42327997': 59,
 'CH52875123': 5,
 'EC88238269': 5,
 'CD56937947': 116,
 'EA58274528': 23,
 'BD69905557': 54,
 'BI09934947': 28,
 'BF07405031': 29,
 'BG29404827': 15,
 'FB7270916': 128,
 'DD25600533': 73,
 'DI87778002': 12,
 'DB10482182': 61,
 'FG9826712': 103,
 'HF47671': 8,
 'DJ25613292': 20,
 'CA48999821': 19,
 'DB23500032': 195,
 'CG37352610': 15,
 'BA79339663': 1429,
 'DG46961870': 92,
 'CC71261888': 12,
 'DI16191065': 140,
 'DJ86193980': 90,
 'CB93694254': 47,
 'CE7769793': 83,
 'BJ18945522': 30,
 'CB87183134': 56,
 'CJ71889961': 85,
 'CF56133275': 20,
 'CD93365897': 7,
 'DH09066919': 21,
 'EB79324570': 23,
 'BH74655165': 31,
 'CB64973127': 146,
 'FC7523423': 16,
 'DB31909049': 48,
 'BC92184871': 25,
 

In [6]:
events_attended_bt = {}
events_maybe_bt = {}
events_invited_bt = {}
events_notgoing_bt = {}
for tid, row in tour_convoy_full.items():
    people_attended = row[0].split()
    people_maybe = row[1].split()
    people_invited = row[2].split()
    people_notgoing = row[3].split()
    for bid in people_attended:
        events_attended_bt[bid] = events_attended_bt.get(bid,[]) + [tid]
    for bid in people_maybe:
        events_maybe_bt[bid] = events_maybe_bt.get(bid,[]) + [tid]
    for bid in people_invited:
        events_invited_bt[bid] = events_invited_bt.get(bid,[]) + [tid]
    for bid in people_notgoing:
        events_notgoing_bt[bid] = events_notgoing_bt.get(bid,[])+ [tid]
        
events_attended_friends_bt = {}
events_maybe_friends_bt = {}
events_invited_friends_bt = {}
events_notgoing_friends_bt = {}
no_friends = {}
for bid, row in tqdm(biker_net.items()):
    friends = row[0].split()
    no_friends[bid] = len(friends)
    for friend in friends:
        for tid in events_attended_bt.get(friend,[]):            
            events_attended_friends_bt[(bid,tid)] = events_attended_friends_bt.get((bid,tid), 0) + 1
        for tid in events_maybe_bt.get(friend,[]):
            events_maybe_friends_bt[(bid,tid)] = events_maybe_friends_bt.get((bid,tid), 0) + 1
        for tid in events_invited_bt.get(friend,[]):
            events_invited_friends_bt[(bid,tid)] = events_invited_friends_bt.get((bid,tid), 0) + 1
        for tid in events_notgoing_bt.get(friend,[]):
            events_notgoing_friends_bt[(bid,tid)] = events_notgoing_friends_bt.get((bid,tid), 0) + 1


100%|██████████| 2231/2231 [00:06<00:00, 331.80it/s]


In [7]:
tours_full = pd.read_csv("Data/tours.csv").values
len_tf = tours_full.shape[0]
np.random.seed(42)
tours_full = tours_full[np.random.choice(len_tf, len_tf//10)]
tours_full.shape

(313797, 110)

In [8]:
#Cluster word frequencies using K Means
#word_count = tours_full.loc[:, "w1":"w_other":1].values
word_count = []
for row in tqdm(tours_full):
    w = []
    for i in range(9, 110):
        w.append(int(row[i]))
    word_count.append(w)
    
word_count = np.array(word_count)
 
kmeans = KMeans(n_clusters=30, random_state=0).fit(word_count)
#kmeans = MiniBatchKMeans(n_clusters=30, batch_size = 10, random_state=0).fit(tours_full)
#tours_full = None

100%|██████████| 313797/313797 [00:08<00:00, 37392.28it/s]


In [9]:
word_count.shape

(313797, 101)

In [10]:
#Selecting top countries for bikers and putting rest as others
countries = {}
for bid, row in biker.items():
    countries[row[1]] = countries.get(row[1], 0) + 1
    
countries_list = list(countries.keys())
countries_list.sort(key = lambda x : countries[x], reverse = True)
top_10_countries = countries_list[:10]
country_val = {}
for i, country in enumerate(top_10_countries):
    country_val[country] = i
    
#Selecting top countries for tours and putting rest as others
tour_countries = {}
for bid, row in tours.items():
    tour_countries[row[5]] = tour_countries.get(row[5], 0) + 1
    
tour_countries_list = list(tour_countries.keys())
tour_countries_list.sort(key = lambda x : tour_countries[x], reverse = True)
tour_top_10_countries = tour_countries_list[:10]
tour_country_val = {}
for i, country in enumerate(tour_top_10_countries):
    tour_country_val[country] = i    
    
#Selecting top cities for tours and putting rest as others
tour_cities = {}
for bid, row in tours.items():
    tour_cities[row[2]] = tour_cities.get(row[2], 0) + 1
    
tour_cities_list = list(tour_cities.keys())
tour_cities_list.sort(key = lambda x : tour_cities[x], reverse = True)
tour_top_10_cities = tour_cities_list[:10]
tour_city_val = {}
for i, city in enumerate(tour_top_10_cities):
    tour_city_val[city] = i    

#Selecting top languages for bikers and putting rest as others
langs = {}
for bid, row in biker.items():
    langs[row[1]] = langs.get(row[0], 0) + 1
    
langs_list = list(langs.keys())
langs_list.sort(key = lambda x : langs[x], reverse = True)
top_10_langs = langs_list[:10]
lang_val = {}
for i, lang in enumerate(top_10_langs):
    lang_val[lang] = i

top_10_langs

['ID', 'ES', 'FR', 'KR', 'US', 'AR', 'GB', 'LA', 'GE', 'CA']

In [11]:
lldf = pd.read_csv("Data/locations.csv")
bdf = pd.read_csv("Data/bikers.csv")
ladf = pd.DataFrame()
ladf['time_zone'] = bdf['time_zone']
ladf['latitude'] = lldf['latitude']
lodf = pd.DataFrame()
lodf['time_zone'] = bdf['time_zone']
lodf['longitude'] = lldf['longitude']
tz_mod_lat  = ladf.groupby('time_zone').apply(lambda x: x['latitude'].value_counts().idxmax())
tz_mod_long = lodf.groupby('time_zone').apply(lambda x: x['longitude'].value_counts().idxmax())

for bid, row in biker.items():
    if row[7] == '0' and row[6] in tz_mod_lat:
        biker[bid][7] = tz_mod_lat[row[6]]
    if row[8] == '0' and row[6] in tz_mod_long:
        biker[bid][8] = tz_mod_long[row[6]]     

In [12]:
top_10_loc = { 
                 'ID': [-2.4833826, 117.8902853],
                 'US': [39.7837304, -100.4458825],
                 'GB': [54.7023545, -3.2765753],
                 'LA': [20.0171109, 103.378253],
                 'ES': [39.3262345, -4.8380649],
                 'FR': [46.603354, 1.8883335],
                 'GE': [32.3293809, -83.1137366],
                 'AR': [-34.9964963, -64.9672817],
                 'CN': [35.000074, 104.999927],
                 'RU': [64.6863136, 97.7453061]
}
for bid, row in biker.items():
    if row[7] == '0' and row[1] in top_10_loc:
        biker[bid][7] = top_10_loc[row[1]][0]
    if row[8] == '0' and row[1] in top_10_loc:
        biker[bid][8] = top_10_loc[row[1]][1]     


In [13]:
biker_full = { key: row for key, *row in csv.reader(open("Data/bikers.csv", 'r'))}
print(biker_full.pop('biker_id'))

['language_id', 'location_id', 'bornIn', 'gender', 'member_since', 'area', 'time_zone']


In [14]:
biker_full_lat = {}
biker_full_long = {}
ladf['biker_id'] =bdf['biker_id']
lodf['biker_id'] =bdf['biker_id']

for index, row in ladf.iterrows():
    bid = row['biker_id']
#     if not pd.isna(row['latitude']):
#         biker_full_lat[bid] = row['latitude']
    if pd.isna(row['latitude']) and (row['time_zone'] in tz_mod_lat):
        biker_full_lat[bid] = tz_mod_lat[row['time_zone']]
    elif biker_full[bid][1] in top_10_loc:
        biker_full_lat[bid] = top_10_loc[biker_full[bid][1]][0]

for index, row in lodf.iterrows():
    bid = row['biker_id']
#     if not pd.isna(row['longitude']):
#         biker_full_long[bid] = row['longitude']
    if pd.isna(row['longitude']) and (row['time_zone'] in tz_mod_long):
        biker_full_long[bid] = tz_mod_long[row['time_zone']]
    elif biker_full[bid][1] in top_10_loc:
        biker_full_long[bid] = top_10_loc[biker_full[bid][1]][1]

In [15]:
cc_set = {
            'LA': [20.0171109, 103.378253],
            'RU': [64.6863136, 97.7453061],
            'ID': [-2.4833826, 117.8902853],
            'LT': [55.3500003, 23.7499997],
            'VN': [13.2904027, 108.4265113],
            'CN': [35.000074, 104.999927],
            'BR': [-10.3333333, -53.2],
            'ES': [39.3262345, -4.8380649],
            'FR': [46.603354, 1.8883335],
            'UD': [39.7837304, -100.4458825],
            'US': [39.7837304, -100.4458825],
            'GE': [32.3293809, -83.1137366],
            'GB': [54.7023545, -3.2765753],
            'TW': [23.9739374, 120.9820179],
            'CA': [61.0666922, -107.9917071]
        }

com_coun = {**cc_set, **top_10_loc}

def get_nan_count():
    nan_count = 0
    for tid, row in tours.items():
        if row[6] == '' or row[7] == '':
            nan_count+=1
    return nan_count

def get_friend_org(org):
    for bid, row in biker_net.items():
        friends = row[0].split()
        if org in friends:
            return bid
    return ''


def repl_missing_tour_loc1():
    bikerlalist = set(biker_full_lat)
    bikerlolist = set(biker_full_long)
    for tid, row in tours.items():
        if row[6] == '':
            att_list = tour_convoy_full[tid][0].split()
            att_list = bikerlalist.intersection(att_list)
            inv_list = tour_convoy_full[tid][2].split()
            inv_list = bikerlalist.intersection(inv_list)
            bc_list = []
            for bi in att_list:
                bc_list.append(biker_full_lat[bi])
            for bi in inv_list:
                bc_list.append(biker_full_lat[bi])
            if len(bc_list) > 0:
                most_common_la = max(set(bc_list), key=bc_list.count)
                tours[tid][6] = most_common_la
        if row[7] == '':
            att_list = tour_convoy_full[tid][0].split()
            att_list = bikerlolist.intersection(att_list)
            inv_list = tour_convoy_full[tid][2].split()
            inv_list = bikerlolist.intersection(inv_list)
            bc_list = []
            for bi in att_list:
                bc_list.append(biker_full_long[bi])
            for bi in inv_list:
                bc_list.append(biker_full_long[bi])
            if len(bc_list) > 0:
                most_common_lo = max(set(bc_list), key=bc_list.count)
                tours[tid][7] = most_common_lo

def repl_missing_tour_loc2():
    for tid, row in tqdm(tours.items()):
        fren = ''
        if row[6] == '' or row[7] == '':
            fren = get_friend_org(row[0])
        if row[6] == '':
            if fren not in biker_full_lat:
                continue
            #print("hi")
            tours[tid][6] = biker_full_lat[fren]
        if row[7] == '':
            if fren not in biker_full_long:
                continue
            tours[tid][7] = biker_full_long[fren]

print(get_nan_count())

repl_missing_tour_loc1()
print(get_nan_count())

repl_missing_tour_loc2()
print(get_nan_count())

latlist = []
longlist = []
for tid, row in tours.items():
    if row[6] != '':
        latlist.append((float(row[6])))
    if row[7] != '':
        longlist.append((float(row[7])))
        
latmed = np.median(np.array(latlist))
longmed = np.median(np.array(longlist))

for tid, row in tours.items():
    if row[6] == '':
        tours[tid][6] = latmed
    if row[7] == '':
        tours[tid][7] = longmed
get_nan_count()

3770


  0%|          | 2/9343 [00:00<10:14, 15.20it/s]

2479


100%|██████████| 9343/9343 [05:49<00:00, 26.73it/s]

2296


0

In [16]:
#Number of tours each person has in train/test set
train_count = {}
test_count = {}
like_count = {}
dislike_count = {}
like_tour_count = {}
dislike_tour_count = {}
train_tour_count = {}
test_tour_count = {}

for index, row in train.iterrows():
    train_count[row["biker_id"]]   = train_count.get(row["biker_id"],0)+1
    like_count[row["biker_id"]]    = like_count.get(row["biker_id"],0)+row["like"]
    dislike_count[row["biker_id"]] = dislike_count.get(row["biker_id"],0)+row["dislike"]
    train_tour_count[row["tour_id"]]   = train_tour_count.get(row["tour_id"],0)+1
    like_tour_count[row["tour_id"]]    = like_tour_count.get(row["tour_id"],0)+row["like"]
    dislike_tour_count[row["tour_id"]] = dislike_tour_count.get(row["tour_id"],0)+row["dislike"]
    
    
for index, row in test.iterrows():
    test_count[row["biker_id"]] = test_count.get(row["biker_id"],0)+1
    test_tour_count[row["tour_id"]]   = test_tour_count.get(row["tour_id"],0)+1

In [17]:
events_attended_dist = {}
events_maybe_dist = {}
events_invited_dist = {}
events_notgoing_dist = {}
for tid, row in tour_convoy_full.items():
    if tid == 'tour_id' or tid not in tours:
        continue
    people_attended = row[0].split()
    people_maybe = row[1].split()
    people_invited = row[2].split()
    people_notgoing = row[3].split()
    for bid in people_attended:
        events_attended_dist[bid] = events_attended_dist.get(bid, np.zeros(101)) + np.array(list(map(lambda x: int(x), tours[tid][8:109])))
    for bid in people_maybe:
        events_maybe_dist[bid] = events_maybe_dist.get(bid, np.zeros(101)) + np.array(list(map(lambda x: int(x), tours[tid][8:109])))
    for bid in people_invited:
        events_invited_dist[bid] = events_invited_dist.get(bid, np.zeros(101)) + np.array(list(map(lambda x: int(x), tours[tid][8:109])))
    for bid in people_notgoing:
        events_notgoing_dist[bid] = events_notgoing_dist.get(bid, np.zeros(101)) + np.array(list(map(lambda x: int(x), tours[tid][8:109])))
        
events_attended_friends_dist = {}
events_maybe_friends_dist = {}
events_invited_friends_dist = {}
events_notgoing_friends_dist = {}
for bid, row in biker_net.items():
    friends = row[0].split()
    for friend in friends:
        events_attended_friends_dist[bid] = events_attended_friends_dist.get(bid, np.zeros(101)) + events_attended_dist.get(friend, np.zeros(101))
        events_maybe_friends_dist[bid] = events_maybe_friends_dist.get(bid, np.zeros(101)) + events_maybe_dist.get(friend, np.zeros(101))
        events_invited_friends_dist[bid] = events_invited_friends_dist.get(bid, np.zeros(101)) + events_invited_dist.get(friend, np.zeros(101))
        events_notgoing_friends_dist[bid] = events_notgoing_friends_dist.get(bid, np.zeros(101)) + events_notgoing_dist.get(friend, np.zeros(101))
        
for bid in events_attended_dist:
    events_attended_dist[bid] /= (events_attended[bid] + 1)
    
for bid in events_maybe_dist:
    events_maybe_dist[bid] /= (events_maybe[bid] + 1)
    
for bid in events_invited_dist:
    events_invited_dist[bid] /= (events_invited[bid] + 1)
    
for bid in events_notgoing_dist:
    events_notgoing_dist[bid] /= (events_notgoing[bid] + 1)
    
for bid in events_attended_friends_dist:
    events_attended_friends_dist[bid] /= (events_attended_friends[bid] + 1)
    
for bid in events_maybe_friends_dist:
    events_maybe_friends_dist[bid] /= (events_maybe_friends[bid] + 1)
    
for bid in events_invited_friends_dist:
    events_invited_friends_dist[bid] /= (events_invited_friends[bid] + 1)
    
for bid in events_notgoing_friends_dist:
    events_notgoing_friends_dist[bid] /= (events_notgoing_friends[bid] + 1)
    
len(events_attended_friends_dist)

2231

In [18]:
lang_attended = {}
lang_maybe = {}
lang_invited = {}
lang_notgoing = {}
lat_long = []
for bid, row in biker.items():
    lat_long.append([row[7],row[8]])
    
lat_long=np.array(lat_long)
kmeans_ll = KMeans(n_clusters=30, random_state=0).fit(lat_long)
kmeans_ll1 = KMeans(n_clusters=15, random_state=0).fit(lat_long)

llclus_attended = {}
llclus1_attended = {}
llclus_maybe = {}
llclus1_maybe = {}
llclus_invited = {}
llclus1_invited = {}
llclus_notgoing = {}
llclus1_notgoing = {}
for tid, row in tour_convoy_full.items():
    if tid == 'tour_id' or tid not in tours:
        continue
    people_attended = row[0].split()
    people_maybe    = row[1].split()
    people_invited  = row[2].split()
    people_notgoing = row[3].split()
    for bid in people_attended:
        if bid not in biker:
            continue
        lang_attended[(biker[bid][0], tid)] = lang_attended.get((biker[bid][0], tid), 0) + 1
        clu = kmeans_ll.predict(np.array([biker[bid][7:9]]))[0]
        clu1 = kmeans_ll1.predict(np.array([biker[bid][7:9]]))[0]
        llclus_attended[(clu,tid)] = llclus_attended.get((clu,tid),0) + 1
        llclus1_attended[clu1] = llclus1_attended.get(clu1,0) + 1
    for bid in people_maybe:
        if bid not in biker:
            continue
        lang_maybe[(biker[bid][0], tid)] = lang_maybe.get((biker[bid][0], tid), 0) + 1
        clu = kmeans_ll.predict(np.array([biker[bid][7:9]]))[0]
        clu1 = kmeans_ll1.predict(np.array([biker[bid][7:9]]))[0]
        llclus_maybe[(clu, tid)] = llclus_maybe.get((clu,tid),0) + 1
        llclus1_maybe[clu1] = llclus1_maybe.get(clu1,0) + 1
    for bid in people_invited:
        if bid not in biker:
            continue
        lang_invited[(biker[bid][0], tid)] = lang_invited.get((biker[bid][0], tid), 0) + 1
        clu = kmeans_ll.predict(np.array([biker[bid][7:9]]))[0]
        clu1 = kmeans_ll1.predict(np.array([biker[bid][7:9]]))[0]
        llclus_invited[(clu, tid)] = llclus_invited.get((clu,tid),0) + 1
        llclus1_invited[clu1] = llclus1_invited.get(clu1,0) + 1
    for bid in people_notgoing:
        if bid not in biker:
            continue
        lang_notgoing[(biker[bid][0], tid)] = lang_notgoing.get((biker[bid][0], tid), 0) + 1
        clu = kmeans_ll.predict(np.array([biker[bid][7:9]]))[0]
        clu1 = kmeans_ll1.predict(np.array([biker[bid][7:9]]))[0]
        llclus_notgoing[(clu,tid)] = llclus_notgoing.get((clu,tid),0) + 1
        llclus1_notgoing[clu1] = llclus1_notgoing.get(clu1,0) + 1
        
bt_attended = {}
bt_maybe = {}
bt_invited = {}
bt_notgoing = {}
for tid, row in tour_convoy_full.items():
    if tid == 'tour_id' or tid not in tours:
        continue
    people_attended = row[0].split()
    people_maybe    = row[1].split()
    people_invited  = row[2].split()
    people_notgoing = row[3].split()
    for bid in people_attended:
        bt_attended[(bid, tid)] = 1
    for bid in people_maybe:
        bt_maybe[(bid, tid)] = 1
    for bid in people_invited:
        bt_invited[(bid, tid)] = 1
    for bid in people_notgoing:
        bt_notgoing[(bid, tid)] = 1

In [19]:
def get_like_count(bid):
    return like_count.get(bid,0)

def get_dislike_count(bid):
    return dislike_count.get(bid,0)

def get_like_tour_count(tid):
    return like_tour_count.get(tid,0)

def get_dislike_tour_count(tid):
    return dislike_tour_count.get(tid,0)

def get_no_friends_count(bid):
    return no_friends.get(bid,0)

def get_train_count(bid):
    return train_count.get(bid,0)

def get_test_count(bid):
    return test_count.get(bid,0)

def get_train_tour_count(tid):
    return train_tour_count.get(tid,0)

def get_test_tour_count(tid):
    return test_tour_count.get(tid,0)

def get_country(bid):
    return country_val.get(biker[bid][1], 10)

def get_lang(bid):
    return lang_val.get(biker[bid][0].upper(), 10)

def get_tour_country(tid):
    return tour_country_val.get(tours[tid][5], 10)

def get_tour_city(tid):
    return tour_city_val.get(tours[tid][2], 10)

def get_delta1(bid, tid, timestamp):
    return (abs(datetime.strptime(timestamp[:10], "%d-%m-%Y") - 
                datetime.strptime(tours[tid][1], "%d-%m-%Y")).total_seconds())/1e7

def get_delta2(bid, tid, timestamp):
    return (abs(datetime.strptime(timestamp[:10], "%d-%m-%Y") - 
                datetime.strptime(biker[bid][4], "%d-%m-%Y")).total_seconds())/1e7

def get_delta3(bid, tid):
    return (abs(datetime.strptime(biker[bid][4], "%d-%m-%Y") - 
                datetime.strptime(tours[tid][1], "%d-%m-%Y")).total_seconds())/1e7

def get_distance(bid, tid):
    return distance((biker[bid][7], biker[bid][8]), (tours[tid][6], tours[tid][7])).miles

def get_llclus(bid,tid):
    clus = kmeans_ll.predict(np.array([biker[bid][7:9]]))[0]
    return [llclus_attended.get((clus,tid),0),llclus_maybe.get((clus,tid),0),llclus_invited.get((clus,tid),0),llclus_notgoing.get((clus,tid),0)]

def get_llclus1(bid):
    clus1 = kmeans_ll1.predict(np.array([biker[bid][7:9]]))[0]
    return [llclus1_attended.get(clus1,0), llclus1_maybe.get(clus1,0),llclus1_invited.get(clus1,0),llclus1_notgoing.get(clus1,0)]

def get_top10_wc(tid):
    row = tours[tid]
    w = []
    for i in range(8, 109):
        w.append(int(row[i]))
    w = np.array(w)
    return kmeans.predict(np.array([w]))[0]

def get_sum_wc(tid):
    row = tours[tid]
    s = 0
    for i in range(8,18):
        s += int(row[i])
    return s

def get_gender(bid):
    return biker[bid][3] == 'male'
    
def get_age(bid):
    by = (biker[bid][2])
    if by == None or by == 'None':
        by = 1990
    return (2020 - int(by))

def get_friend_with_org(bid, tid):
    org = tours[tid][0]
    frens = biker_net[bid][0].split()
    return org in frens

def get_ymdw(tobj):
    tim = datetime.strptime(tobj, "%d-%m-%Y")
    return [tim.year, tim.month, tim.day, tim.weekday()]

def get_timezone(bid):
    tz = biker[bid][6]
    if tz == '':
        return 7.0
    return int(tz)/60

def get_sim_a(bid, tid):
    v1 = events_attended_dist.get(bid, None)
    if v1 is None:
        return 0
    v2 = np.array(list(map(lambda x: int(x), tours[tid][8:109])))
    a = np.linalg.norm(v1)
    b = np.linalg.norm(v2)
    if a == 0 or b == 0 :
        return 0
    return np.dot(v1, v2)/(a*b)

def get_sim_m(bid, tid):
    v1 = events_maybe_dist.get(bid, None)
    if v1 is None:
        return 0
    v2 = np.array(list(map(lambda x: int(x), tours[tid][8:109])))
    a = np.linalg.norm(v1)
    b = np.linalg.norm(v2)
    if a == 0 or b == 0 :
        return 0
    return np.dot(v1, v2)/(a*b)

def get_sim_i(bid, tid):
    v1 = events_invited_dist.get(bid, None)
    if v1 is None:
        return 0
    v2 = np.array(list(map(lambda x: int(x), tours[tid][8:109])))
    a = np.linalg.norm(v1)
    b = np.linalg.norm(v2)
    if a == 0 or b == 0 :
        return 0
    return np.dot(v1, v2)/(a*b)

def get_sim_n(bid, tid):
    v1 = events_notgoing_dist.get(bid, None)
    if v1 is None:
        return 0
    v2 = np.array(list(map(lambda x: int(x), tours[tid][8:109])))
    a = np.linalg.norm(v1)
    b = np.linalg.norm(v2)
    if a == 0 or b == 0 :
        return 0
    return np.dot(v1, v2)/(a*b)

def get_sim_af(bid, tid):
    v1 = events_attended_friends_dist.get(bid, None)
    if v1 is None:
        return 0
    v2 = np.array(list(map(lambda x: int(x), tours[tid][8:109])))
    a = np.linalg.norm(v1)
    b = np.linalg.norm(v2)
    if a == 0 or b == 0 :
        return 0
    return np.dot(v1, v2)/(a*b)

def get_sim_mf(bid, tid):
    v1 = events_maybe_friends_dist.get(bid, None)
    if v1 is None:
        return 0
    v2 = np.array(list(map(lambda x: int(x), tours[tid][8:109])))
    a = np.linalg.norm(v1)
    b = np.linalg.norm(v2)
    if a == 0 or b == 0 :
        return 0
    return np.dot(v1, v2)/(a*b)

def get_sim_if(bid, tid):
    v1 = events_invited_friends_dist.get(bid, None)
    if v1 is None:
        return 0
    v2 = np.array(list(map(lambda x: int(x), tours[tid][8:109])))
    a = np.linalg.norm(v1)
    b = np.linalg.norm(v2)
    if a == 0 or b == 0 :
        return 0
    return np.dot(v1, v2)/(a*b)

def get_sim_nf(bid, tid):
    v1 = events_notgoing_friends_dist.get(bid, None)
    if v1 is None:
        return 0
    v2 = np.array(list(map(lambda x: int(x), tours[tid][8:109])))
    a = np.linalg.norm(v1)
    b = np.linalg.norm(v2)
    if a == 0 or b == 0 :
        return 0
    return np.dot(v1, v2)/(a*b)

fraction_in = {}
fraction_in1 = {}
fraction_in2 = {}
total_attendees = {}
total_maybe = {}
total_invited = {}
total_notgoing = {}
deltas = {}

for tid, row in tour_convoy.items():
    not_going = len(row[3].split())
    invited = len(row[2].split())
    maybe = len(row[1].split())
    attend = len(row[0].split())
    fraction_in[tid]  = (not_going)/(invited+1)
    fraction_in1[tid] = (attend)/(invited+1)
    fraction_in2[tid] = (maybe)/(invited+1)
    total_attendees[tid] = attend
    total_maybe[tid] = maybe
    total_invited[tid] = invited
    total_notgoing[tid] = maybe

for index, row in train.iterrows():
    a = row["biker_id"]
    b = row["tour_id"]
    c = row["timestamp"]
    curr_delta = get_delta1(a,b,c)
    if a not in deltas:
        deltas[a] = curr_delta
    else:
        deltas[a] = min(deltas[a], curr_delta)
        
for index, row in test.iterrows():
    a = row["biker_id"]
    b = row["tour_id"]
    c = row["timestamp"]
    curr_delta = get_delta1(a,b,c)
    if a not in deltas:
        deltas[a] = curr_delta
    else:
        deltas[a] = min(deltas[a], curr_delta)

In [20]:
cluster_attend = {}
cluster_maybe = {}
cluster_invited = {}
cluster_notgoing = {}
for tid, row in tours.items():
    w = []
    for i in range(8, 109):
        w.append(int(row[i]))
    w = np.array(w)
    clus = kmeans.predict(np.array([w]))[0]
    cluster_attend[tid]   = cluster_attend.get(tid, 0) + total_attendees[tid]
    cluster_maybe[tid]    = cluster_maybe.get(tid, 0) + total_maybe[tid]
    cluster_invited[tid]  = cluster_invited.get(tid, 0) + total_invited[tid]
    cluster_notgoing[tid] = cluster_notgoing.get(tid, 0) + total_notgoing[tid]

cluster_attend_bt   = {}
cluster_maybe_bt    = {}
cluster_invited_bt  = {}
cluster_notgoing_bt = {}    
for bid, row in biker.items():
    for tid in events_attended_bt.get(bid,[]):
        if tid not in tours:
            continue
        w = list(map(int, tours[tid][8:109])) 
        w = np.array(w)
        clus = kmeans.predict(np.array([w]))[0]
        cluster_attend_bt[(bid,clus)] = cluster_attend_bt.get((bid,clus), 0) + 1
    for tid in events_maybe_bt.get(bid,[]):
        if tid not in tours:
            continue
        w = list(map(int, tours[tid][8:109])) 
        w = np.array(w)
        clus = kmeans.predict(np.array([w]))[0]
        cluster_maybe_bt[(bid,clus)] = cluster_maybe_bt.get((bid,clus), 0) + 1
    for tid in events_invited_bt.get(bid,[]):
        if tid not in tours:
            continue
        w = list(map(int, tours[tid][8:109])) 
        w = np.array(w)
        clus = kmeans.predict(np.array([w]))[0]
        cluster_invited_bt[(bid,clus)] = cluster_invited_bt.get((bid,clus), 0) + 1
    for tid in events_notgoing_bt.get(bid,[]):
        if tid not in tours:
            continue
        w = list(map(int, tours[tid][8:109])) 
        w = np.array(w)
        clus = kmeans.predict(np.array([w]))[0]
        cluster_notgoing_bt[(bid,clus)] = cluster_notgoing_bt.get((bid,clus), 0) + 1
        
def get_clus_bt(bid,tid):
    w = list(map(int, tours[tid][8:109])) 
    w = np.array(w)
    clus = kmeans.predict(np.array([w]))[0]
    return [cluster_attend_bt.get((bid,clus),0),cluster_maybe_bt.get((bid,clus),0),cluster_invited_bt.get((bid,clus),0),cluster_notgoing_bt.get((bid,clus),0)]

In [24]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
dtMatrix = []
tid_to_ind = {}
for index, (tid, row) in enumerate(tours.items()):
    w = list(map(int, tours[tid][8:109]))
    dtMatrix.append(w)
    tid_to_ind[tid] = index
dtMatrix = np.array(dtMatrix)
tfidf = TfidfTransformer()
tfidfMatrix = tfidf.fit_transform(dtMatrix).toarray()
svd = TruncatedSVD(n_components = 10, random_state = 0)
svdMatrix = svd.fit_transform(tfidfMatrix)

In [25]:
bikers_useful = pd.read_csv("Data/bikers_useful.csv")
tours_useful = pd.read_csv("Data/tours_useful.csv")

train_x=pd.merge(train,bikers_useful,on='biker_id',how='left')
train_x=pd.merge(train_x,tours_useful,on='tour_id',how='left')

Y=train_x['like']
train_x.drop(columns=['like','dislike', 'biker_id_y'],inplace=True)

test_x=pd.merge(test,bikers_useful,on='biker_id',how='left')
test_x=pd.merge(test_x,tours_useful,on='tour_id',how='left')
test_x.drop(columns=['biker_id_y'],inplace=True)

for col in ['gender', 'country', 'city', 'state', 'pincode', 'area']:
    train_x[col].fillna('',inplace=True)
    test_x[col].fillna('',inplace=True)

In [26]:
d = 0
def make_feature(bid, tid, timestamp, invited):
    f = [0.0]*85
    
    f[0] = get_delta1(bid, tid, timestamp)
    f[1] = fraction_in[tid]
    f[2] = total_attendees[tid]
    f[3] = get_distance(bid, tid)
    f[4] = events_attended_friends[bid]
    f[5] = events_maybe_friends[bid]
    f[6] = events_notgoing_friends[bid]
    f[7] = 1 if f[0] == 0 else deltas[bid]/f[0]
    f[8] = total_maybe[tid]
    
    f[9] = int(get_country(bid))               #cat
    
#     f[10] = int(get_top10_wc(tid))             #cat
    
#     f[11] = get_train_count(bid)
#     f[12] = get_test_count(bid)
    f[13] = get_delta2(bid, tid, timestamp)
    f[14] = get_delta3(bid, tid)
    f[15] = total_notgoing[tid]
    f[16] = get_friend_with_org(bid, tid)
    f[17] = get_age(bid)
    
    t1 = get_ymdw(timestamp[:10])
    t1 += get_ymdw(biker[bid][4])
    t1 += get_ymdw(tours[tid][1])
    for j in range(12):
        f[18+j] = t1[j]                    #cat
    
    f[30] = get_tour_country(tid)          #cat   
    f[31] = get_tour_city(tid)             #cat     
    f[32] = get_lang(bid)                  #cat
    
    f[33] = get_no_friends_count(bid)
    f[34] = get_timezone(bid)
    f[35] = get_sum_wc(tid)
    f[36] = (t1[3] in [4,5,6])             #cat
    f[37] = (t1[7] in [4,5,6])             #cat
    f[38] = (t1[11] in [4,5,6])            #cat
    
#     f[39] = get_like_count(bid)
#     f[40] = get_dislike_count(bid)
    
#     f[41] = get_train_tour_count(tid)
    
    f[42] = events_attended.get(bid,0)
    f[43] = events_maybe.get(bid,0)
    f[44] = events_notgoing.get(bid,0)
    
    for i in range(10):
        f[45+i] = svdMatrix[tid_to_ind[tid]][i]
#     f[45] = get_test_tour_count(tid)
    
#     f[46] = get_sim_a(bid, tid)
#     f[47] = get_sim_m(bid, tid)
#     f[48] = get_sim_n(bid, tid)
#     f[49] = get_sim_af(bid, tid)
#     f[50] = get_sim_mf(bid, tid)
#     f[51] = get_sim_nf(bid, tid)
    
#     f[52] = t1[1]//4                       #cat
#     f[53] = t1[5]//4                       #cat
#     f[54] = t1[9]//4                       #cat
    
#     f[55] = int(t1[9] == 12)               #cat
   
#     f[56] = cluster_attend[tid]  
#     f[57] = cluster_notgoing[tid]
#     f[58] = cluster_maybe[tid]   
    
#     f[59] = lang_attended.get((biker[bid][0], tid),0)
#     f[60] = lang_notgoing.get((biker[bid][0], tid),0)
#     f[61] = lang_maybe.get((biker[bid][0], tid),0)

#     llclu1 = get_llclus1(bid)
#     f[62] = llclu1[0]
#     f[63] = llclu1[1]
#     f[64] = llclu1[2]
#     f[65] = llclu1[3]
    
    f[66] = events_attended_friends_bt.get((bid,tid),0)
    f[67] = events_maybe_friends_bt.get((bid,tid),0)
    f[68] = events_invited_friends_bt.get((bid,tid),0)
    f[69] = events_notgoing_friends_bt.get((bid,tid),0)
    
#     f[70] = lang_invited.get((biker[bid][0], tid),0)
#     f[71] = cluster_invited[tid]   
    
#     f[72] = get_sim_i(bid, tid)
#     f[73] = get_sim_if(bid, tid)
    
    f[74] = events_invited.get(bid,0)
    f[75] = events_invited_friends[bid]
    f[76] = total_invited[tid]
    
#     llclu = get_llclus(bid,tid)
#     f[77] = llclu[0]
#     f[78] = llclu[1]
#     f[79] = llclu[2]
#     f[80] = llclu[3]
    
#     clu_bt = get_clus_bt(bid,tid)
#     f[81] = clu_bt[0]
#     f[82] = clu_bt[1]
#     f[83] = clu_bt[2]
#     f[84] = clu_bt[3]

#     latib  = np.radians(float(biker[bid][7]))
#     longib = np.radians(float(biker[bid][8]))
#     f[81] = np.cos(latib)*np.cos(longib)
#     f[82] = np.sin(latib)*np.sin(longib)
#     f[83] = np.sin(latib)
    
    return f

X = []
for index, row in train_x.iterrows():
    a = row["biker_id_x"]
    b = row["tour_id"]
    c = row["timestamp"]
    d = row["invited"]
    curr_l = list(row.values)
    curr_l += make_feature(a, b, c, d)
    X.append(curr_l)
        
X = pd.DataFrame(X)

test_X = []
for index, row in test_x.iterrows():
    a = row["biker_id_x"]
    b = row["tour_id"]
    c = row["timestamp"]
    d = row["invited"]
    curr_l = list(row.values)
    curr_l += make_feature(a, b, c, d)
    test_X.append(curr_l)
        
test_X = pd.DataFrame(test_X)

for c in [3,8,13]:
    X[c] = (pd.to_datetime(X[c]) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
    test_X[c] = (pd.to_datetime(test_X[c]) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

X.drop(columns=[0,1],inplace=True)
test_X.drop(columns=[0,1],inplace=True)

X.head()

,2,3,4,5,6,7,8,9,10,11,...,196,197,198,199,200,201,202,203,204,205
0,0,1328889185,id,ID,1990,male,1328832000,Binjai,480.0,3.606396,...,290,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1328889185,id,ID,1990,male,1328832000,Binjai,480.0,3.606396,...,290,122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1328889185,id,ID,1990,male,1328832000,Binjai,480.0,3.606396,...,290,3844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1328889185,id,ID,1990,male,1328832000,Binjai,480.0,3.606396,...,290,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,1328889185,id,ID,1990,male,1328832000,Binjai,480.0,3.606396,...,290,3814,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
from catboost import cv

cat_features = [0,1,2,3,4,5,6,7,11,12,13,14,15,128,129,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,155,156,157,171,172,173,174,182]
preds = np.zeros(test_X.shape[0])
# for c in cat_features:
#     if (c+2) not in X:
#         continue
#     col_type = X[c+2].dtype
#     if col_type == 'object' or col_type.name == 'category':
#         X[c+2] = X[c+2].astype('category')
#         test_X[c+2] = test_X[c+2].astype('category')
# cv_dataset = Pool(data=X, label=Y,cat_features=cat_features)

# params1 = {"iterations": 1100,
#           "loss_function": "Logloss",
#           "verbose": False,
#           "random_state":0, 
#           "l2_leaf_reg":3.0,
# #           "depth":6,
# #           "learning_rate" :0.05
#          }
# params2 = {"iterations": 1300,
#           "loss_function": "Logloss",
#           "verbose": False,
#           "random_state":0, 
#           "l2_leaf_reg":3.0,
# #           "depth":6,
# #           "learning_rate" :0.05
#          }
# params3 = {"iterations": 1500,
#           "loss_function": "Logloss",
#           "verbose": False,
#           "random_state":0, 
#           "l2_leaf_reg":3.0,
# #           "depth":6,
# #           "learning_rate" :0.05
#          }

# # scores = cv(cv_dataset, params, fold_count=5)
# # print(scores.tail())
clf1 = CatBoostClassifier(**params1)
# clf2 = CatBoostClassifier(**params2)
# clf3 = CatBoostClassifier(**params3)

# from sklearn.metrics import log_loss
# n_bag = 30
# sss = StratifiedShuffleSplit(n_splits=n_bag, test_size=0.1, random_state=0)
# sss.get_n_splits(X, Y)

# print(sss)       
# ct = 0
# bs = []
# for train_index, test_index in sss.split(X, Y):
# #     break
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
#     cv_dataset = Pool(data=X_train, label=y_train,cat_features=cat_features)
#     clf1.fit(cv_dataset)
#     clf2.fit(cv_dataset)
#     clf3.fit(cv_dataset)
#     a = (1-log_loss(y_test, clf1.predict_proba(X_test)[:,1]))
# #     if a > 0.57:
#     print(a,ct)
#     preds += clf1.predict_proba(test_X)[:,1]
#     preds += clf2.predict_proba(test_X)[:,1]
#     preds += clf3.predict_proba(test_X)[:,1]
#     ct+=1
#     bs.append(a)

# df_describe = pd.DataFrame(bs)
# print(df_describe.describe())
# print(ct)
# scores = 1+cross_val_score(clf, X, Y, cv = 5, scoring = 'neg_log_loss')
# print(scores)
# print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

# clf1.fit(cv_dataset)
# preds += clf1.predict_proba(test_X)[:,1]
# ct+=1

NameError: name 'params1' is not defined

In [30]:
for c in cat_features:
    if (c+2) not in X:
        continue
    col_type = X[c+2].dtype
    if col_type == 'object' or col_type.name == 'category':
        X[c+2] = X[c+2].astype('category')
        test_X[c+2] = test_X[c+2].astype('category')

fit_params1={ 
            "metric" : 'auc', 
            'random_state':0,
            'deterministic':True,
            'n_jobs':1,
            'num_leaves':27,
            'n_estimators':300,
#             'max_depth':1000,
            'reg_lambda':0.0000,
            'learning_rate':0.05200,
            'colsample_bytree':0.95,
#             'is_unbalance':True,
#             'scale_pos_weight':1.1,
#             'min_child_samples':21,
#             'subsample_freq':10,
#             'subsample': 0.9
}

fit_params2={ 
            "metric" : 'auc', 
            'random_state':0,
            'deterministic':True,
            'n_jobs':1,
            'num_leaves':27,
            'n_estimators':400,
#             'max_depth':1000,
            'reg_lambda':0.0000,
            'learning_rate':0.05200,
            'colsample_bytree':0.90,
#             'is_unbalance':True,
#             'scale_pos_weight':1.1,
#             'min_child_samples':21,
#             'subsample_freq':10,
#             'subsample': 0.9
}

fit_params3={ 
            "metric" : 'auc', 
            'random_state':0,
            'deterministic':True,
            'n_jobs':1,
            'num_leaves':27,
            'n_estimators':500,
#             'max_depth':1000,
            'reg_lambda':0.0000,
            'learning_rate':0.05200,
#             'is_unbalance':True,
#             'scale_pos_weight':1.1,
#             'min_child_samples':21,
#             'subsample_freq':10,
#             'subsample': 0.9
}

clf1 = LGBMClassifier(
        **fit_params1
    )
clf2 = LGBMClassifier(
        **fit_params2
    )
clf3 = LGBMClassifier(
        **fit_params3
    )

from sklearn.metrics import log_loss
n_bag = 200
sss = StratifiedShuffleSplit(n_splits=n_bag, test_size=0.1, random_state=0)
sss.get_n_splits(X, Y)

print(sss)       
ct = 0
bs = []
for train_index, test_index in sss.split(X, Y):
    break
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    clf1.fit(X_train, y_train)
    clf2.fit(X_train, y_train)
    clf3.fit(X_train, y_train)
    y1 = clf1.predict_proba(X_test)[:,1]
    y2 = clf2.predict_proba(X_test)[:,1]
    y3 = clf3.predict_proba(X_test)[:,1]
    y4 = clf1.predict_proba(test_X)[:,1]
    y5 = clf2.predict_proba(test_X)[:,1]
    y6 = clf3.predict_proba(test_X)[:,1]
    a = (1-log_loss(y_test, (y1+y2+y3)/3))
#     if a > 0.55:
    if ct%5 == 0:
        print(a,ct)
    preds = preds + (y4+y5+y6)
    ct+=1
    bs.append(a)

# df_describe = pd.DataFrame(bs)
# print(df_describe.describe())
# print(ct)
# for nl in [23,25,27,29]:
#     for lr in [0.050,0.051,0.052,0.053]:
# for nl in [27]:
#     for lr in [0.052]:
# for rs in range(10):
#     for csb in range(50):
#         for ni in range(3):
#             fit_params1['random_state']=rs
#             fit_params1['colsample_bytree']=0.64
#             fit_params1['n_estimators'] = (ni + 3)*100
#     #         fit_params1['num_leaves'] = nl
#     #         fit_params1['learning_rate'] = lr
#             clf = LGBMClassifier(**fit_params1)
#             clf.fit(X,Y)
#             preds+=clf.predict_proba(test_X)[:,1]
#             ct+=1
for csb in [0.9,0.95]:
    fit_params1['colsample_bytree']=csb
#     fit_params1['random_state']=rs
    clf1 = LGBMClassifier(**fit_params1)
    clf1.fit(X,Y)
    preds+=clf1.predict_proba(test_X)[:,1]
    ct+=1
#     scores = 1+cross_val_score(clf1, X, Y, cv = 5, scoring = 'neg_log_loss')
#     #print(scores)
#     print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))
#     print(fit_params1['colsample_bytree'])
    
# clf = LGBMClassifier(**fit_params1)
# clf.fit(X,Y)
# preds+=clf.predict_proba(test_X)[:,1]
# ct+=1

scores = 1+cross_val_score(clf1, X, Y, cv = 5, scoring = 'neg_log_loss')
print(scores)
print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))
#clf1.fit(X,Y)

StratifiedShuffleSplit(n_splits=200, random_state=0, test_size=0.1,
            train_size=None)
[0.52321983 0.53180514 0.53660684 0.536189   0.51696211]
Accuracy: 0.5290 (+/- 0.0154)


In [ ]:
#buffer
# import matplotlib.pyplot as plt
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

# # sorted(zip(clf.feature_importances_, X.columns), reverse=True)
# feature_imp = pd.DataFrame(sorted(zip(clf.feature_importances_,X.columns)), columns=['Value','Feature'])

# plt.figure(figsize=(20, 10))
# sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
# plt.title('LightGBM Features (avg over folds)')
# plt.tight_layout()
# plt.show()
# plt.savefig('lgbm_importances-01.png')

# clf.fit(X,Y)
# feat_imp = pd.DataFrame({'importance':clf.feature_importances_})    
# feat_imp['feature'] = X.columns
# feat_imp.sort_values(by='importance', ascending=False, inplace=True)
# feat_imp = feat_imp.iloc[150:]

# feat_imp.sort_values(by='importance', inplace=True)
# feat_imp = feat_imp.set_index('feature', drop=True)
# feat_imp.plot.barh(title='Feat Imp', figsize=(8,40))
# plt.xlabel('Feature Importance Score')
# plt.show()
# from autofeat import FeatureSelector
# fsel = FeatureSelector(verbose=1, problem_type="classification", n_jobs=4)
# new_X = fsel.fit_transform(X, Y)
# data = X
# data['target'] = Y
# corr = data.corr()
# # visualise the data with seaborn
# mask = np.triu(np.ones_like(corr, dtype=np.bool))
# sns.set_style(style = 'white')
# f, ax = plt.subplots(figsize=(25, 20))
# cmap = sns.diverging_palette(10, 250, as_cmap=True)
# sns.heatmap(corr, mask=mask, cmap=cmap, 
#         square=True,
#         linewidths=.5, cbar_kws={"shrink": .5}, ax=ax)

from bayes_opt import BayesianOptimization

def lgb_eval(lambda_l2,lambda_l1,min_child_samples, lr,subsample_freq, subsample, num_leaves=27, max_depth=-1):
    params = {
        #'is_unbalance': True,
        "num_leaves" : int(num_leaves),
        "max_depth" : int(max_depth),
        "reg_alpha" : lambda_l2,
        "reg_lambda" : lambda_l1,
        "n_jobs" : 1,
        "min_child_samples" : int(min_child_samples),
        "learning_rate" : lr,
        #"verbosity" : -1,
        #"force_row_wise" : True,
        "n_estimators":300,
        "random_state" : 0,
         'subsample_freq':int(subsample_freq),
         'subsample': subsample
    }
    cll = LGBMClassifier(**params)
    scores = 1+cross_val_score(cll, X, Y, cv = 4, scoring = 'neg_log_loss')
    return scores.mean()


lgbBO = BayesianOptimization(lgb_eval, {
                                                'lambda_l2': (0.0, 0.0005),
                                                'lambda_l1': (0.0, 0.00005),
                                                'min_child_samples': (15, 25),
                                                'lr' : (0.05, 0.055),
                                                #'max_depth': (5, 75),
                                                'num_leaves': (25, 35),
                                                'subsample_freq':(7,13),
                                                'subsample':(0.85,0.97)
                                                })

lgbBO.maximize(n_iter=40, init_points=5)
print(lgbBO.max)

In [ ]:
raise Exception

In [ ]:
feature_names = ['delta1', 'not_going_ratio', 'attended', 'distance', 'eaf', 'min_delta_ratio', 'invited',
                 'maybe', 'enf', 'emf']
feature_names += ["bcou"+str(i) for i in range(11)]
feature_names += ["wc"+str(i) for i in range(30)]
feature_names += ['train_count', 'test_count', 'delta2', 'delta3', 'not_going', 'gender', 'age', 'fr_with_org']
feature_names += ['yts', 'mts', 'dts', 'wts']
feature_names += ['yms', 'mms', 'dms', 'wms']
feature_names += ['ytd', 'mtd', 'dtd', 'wtd']
feature_names += ["tcou"+str(i) for i in range(11)]
feature_names += ["tcit"+str(i) for i in range(11)]
feature_names += ["lang"+str(i) for i in range(11)]
feature_names += ['tz','sum_wc' 'we1', 'we2', 'we3', 'lc', 'dc', 'no_fri']

X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=13)

#params = { 'max_depth':20, 'n_estimators':201, 'max_features':gbmf, 'random_state':0 }
params = { 'max_depth':20, 'n_estimators':201, 'max_features':rfmf, 'random_state':0 , 'max_samples':0.7}
#params = { 'iterations':250, 'learning_rate':0.1, 'depth':10, 'eval_metric': 'Accuracy', 'random_state':0, 'verbose':100}
#params = { 'estimators' : [('rf',clf0), ('gb',clf1)], 'weights' : weights }

#reg = CatBoostClassifier(**params)
reg = RandomForestClassifier(**params)
#reg = GradientBoostingClassifier(**params)
#reg = VotingClassifier(**params)
reg.fit(X_train, y_train)

'''
mse = mean_squared_error(y_test, reg.predict(X_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

test_score = np.zeros((params['n_estimators'],), dtype=np.float64)
for i, y_pred in enumerate(reg.staged_predict(X_test)):
    test_score[i] = reg.loss_(y_test, y_pred)

fig = plt.figure(figsize=(6, 6))
plt.subplot(1, 1, 1)
plt.title('Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, reg.train_score_, 'b-',
         label='Training Set Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-',
         label='Test Set Deviance')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance')
fig.tight_layout()
plt.show() '''

feature_importance = reg.feature_importances_
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
fig = plt.figure(figsize=(17, 17))
plt.subplot(1, 2, 1)
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, np.array(feature_names)[sorted_idx])
plt.title('Feature Importance (MDI)')

result = permutation_importance(reg, X_test, y_test, n_repeats=10,
                                random_state=42, n_jobs=2)
sorted_idx = result.importances_mean.argsort()
plt.subplot(1, 2, 2)
plt.boxplot(result.importances[sorted_idx].T,
            vert=False, labels=np.array(feature_names)[sorted_idx])
plt.title("Permutation Importance (test set)")
fig.tight_layout()
plt.show()

In [31]:
ts = {}
inv = {}
num_bt = {}
for index, row in test.iterrows():
    ts[(row["biker_id"], row["tour_id"])] = row["timestamp"]
    inv[(row["biker_id"], row["tour_id"])] = row["invited"]
    num_bt[(row["biker_id"], row["tour_id"])] = index
    
# test_Y = clf.predict_proba(test_X)[:,1]
# preds = pd.read_csv("Data/a.csv")
# preds = preds['0']
# ct=40
test_Y = preds/ct
test_Y.shape

(2690,)

In [32]:
train_bikers_set = np.array(train["biker_id"].drop_duplicates())
bikers_out = []
tours_out = []
for biker1 in tqdm(test_bikers_set):
    idx = np.where(biker1==test["biker_id"]) 
    tour = list(test["tour_id"].loc[idx]) # for each unique biker in test data get all the events  
    score = {}
    for tou in tour:
        #fe = make_feature(biker1, tou, ts[(biker1, tou)], inv[(biker1, tou)]).reshape(1,-1)
        #fe = [make_feature(biker1, tou, ts[(biker1, tou)], inv[(biker1, tou)])]
        #fe = scaler.transform(fe)
#         s1 = clf0.predict_proba(fe)[0][1]
#         s2 = clf1.predict_proba(fe)[0][1]
#         #print(s1, s2)
#         rf_wt = 0.5
#         score[tou] = s1*rf_wt + s2*(1-rf_wt)
        #print(clf.predict_proba(fe)[0])
        #s = clf.predict_proba(fe)[0][0]
        
        #s = clf.predict_proba(fe)[0][1]        
        
        s = test_Y[num_bt[(biker1, tou)]]
        #s = test_Y[num_bt[(biker1, tou)]][1]
        score[tou] = s
    #print(score)
    tour.sort(key = lambda x : score[x], reverse = True)
    tour = " ".join(tour) # list to space delimited string
    bikers_out.append(biker1)
    tours_out.append(tour)
    

100%|██████████| 297/297 [00:00<00:00, 984.38it/s]


In [33]:
sample_submission =pd.DataFrame(columns=["biker_id","tour_id"])
sample_submission["biker_id"] = bikers_out
sample_submission["tour_id"] = tours_out
sample_submission.to_csv("output/submission310.csv",index=False) # download this file from /kaggle/working directory
print(sample_submission.shape)
sample_submission.head(10)

(297, 2)


,biker_id,tour_id
0,CG33145288,YU2519340 QX16813281 QP76364848 QR03717384 RQ4...
1,EC61865653,QU48928340 QS25647788 YQ5403718 RW48209109 TQ2...
2,CC23946882,SX31661369 YV890078 RU03553114 QP2460414 UX154...
3,DC82478477,YW217712 SV43004825 QV19013260 TR67083677
4,EC46728549,RQ33183406 XR7394707 VP2394192 WW1676713 RU290...
5,DJ43175229,YP6531778 XP6696202 SX39967033 SX39967033 SW79...
6,EA60379526,SS27788367 SX79292193 QX10747729 QT66528489 SX...
7,HB3889887,RP43455253 SX07386691 WT1936389 QW76393554 QP9...
8,CJ45719859,RU98678508 SS69927458 RQ45977549 SS84785057 SQ...
9,BA17536864,RX05037878 RW31716338 SQ50441220 QY9858305 ST9...


In [ ]:
X_ = X.copy(deep=True)
X_['biker_id'] = train['biker_id'].copy()
X_['tour_id'] = train['tour_id'].copy()
X_train, X_test, y_train, y_test = train_test_split(X_, Y, test_size=0.25, random_state=0, stratify = Y)
cat_features = [0,1,2,3,4,5,6,7,11,12,13,14,15,128,129,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,155,156,157,171,172,173,174,182]
test_bt = X_test[['biker_id', 'tour_id']].copy(deep=True)
test_bt.reset_index(inplace=True)
test_bt.drop(columns=['index'], inplace=True)
X_train.drop(columns=['biker_id', 'tour_id'], inplace=True)
X_test.drop(columns=['biker_id', 'tour_id'], inplace=True)


fit_params = {"iterations": 1100,
          "loss_function": "Logloss",
          "verbose": False,
          "random_state":0, 
          "l2_leaf_reg":3.0,
          "depth":6,
          "learning_rate" :0.05
         }

cv_dataset = Pool(data=X_train, label=y_train,cat_features=cat_features)
clf = CatBoostClassifier(
        **fit_params
    )

clf.fit(cv_dataset)
y_pred = clf.predict_proba(X_test)[:,1]

num_bt = {}
for index, row in test_bt.iterrows():
    num_bt[(row["biker_id"], row["tour_id"])] = index
train_bikers_set = np.array(train["biker_id"].drop_duplicates())
bikers_out = []
tours_out = []
for biker1 in tqdm(set(test_bt["biker_id"])):
    idx = np.where(biker1==test_bt["biker_id"]) 
    tour = list(test_bt["tour_id"].loc[idx]) # for each unique biker in test data get all the events  
    score = {}
    for tou in tour:       
        s = y_pred[num_bt[(biker1, tou)]]
        score[tou] = s
    tour.sort(key = lambda x : score[x], reverse = True)
    tour = " ".join(tour) # list to space delimited string
    bikers_out.append(biker1)
    tours_out.append(tour)

sample_submission =pd.DataFrame(columns=["biker_id","tour_id"])
sample_submission["biker_id"] = bikers_out
sample_submission["tour_id"] = tours_out

def avg_prec(bid, tourlist):
    gtp = 0
    s = 0
    for i, tid in enumerate(tourlist):
        if(train[(train['biker_id']==bid) & (train['tour_id']==tid)].iloc[0][4]==1):
            gtp+=1
            #print('here')
            s+=gtp/(i+1)
    if(gtp>0):
        return s/gtp
    else:
        return 0
    
def meanap(reorderedtest):
    den = 0
    num = 0
    for i in range(reorderedtest.shape[0]):
        r = reorderedtest.iloc[[i]]
        bid = r.iloc[0][0]
        tourlist = r.iloc[0][1].split()
        avg = avg_prec(bid, tourlist)
        if(avg!=0):
            den+=1
        num+=avg
    return num/den

print(meanap(sample_submission))

In [36]:
#np.set_printoptions(formatter={'float': '{: 0.6f}'.format}, linewidth=200, suppress=True)
#pd.DataFrame(X).to_csv("Data/x_1.csv", header=False)
# old_x = pd.read_csv("Data/x.csv", header = None)
# old_x[4].values
#probabs.to_csv("Data/probabs.csv", header=False)
pd.options.display.max_columns = 999
# train_x.head()

# X.to_csv("Data/x_2.csv", header=False, index=False)
# Y.to_csv("Data/y_2.csv", header=False, index=False)
# test_X.to_csv("Data/test_x_2.csv", header=False, index=False)

#X.loc[Y == 1][123].describe()
# for frnd in biker_net['EI3332100'][0].split():
#     if frnd in biker_full_lat and frnd in biker_full_long:
#         print(biker_full_lat[frnd], biker_full_long[frnd])

X.head()


,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205
0,0,1328889185,id,ID,1990,male,1328832000,Binjai,480.0,3.606396,98.489987,1331337600,,,,,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0.00864,1.041667,8,9805.457996,21,10,16,1.000000,2,0,0.0,0.0,0.0,0.0,0.00864,2,False,30,2012,10,2,1,2012,10,2,1,2012,10,3,2,0,0,0,862,8.0,0,False,False,False,0.0,0.0,0.0,0,0,0,0.989033,-0.075436,0.052919,-0.096919,0.000461,-0.036076,-0.019340,0.025858,-0.005333,-0.004627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,3,3,0.0,0.0,0.0,0.0,0,290,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1328889185,id,ID,1990,male,1328832000,Binjai,480.0,3.606396,98.489987,1331337600,Yogyakarta,,,Indonesia,-7.767,110.363,2,0,0,0,2,3,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,0.00864,0.016260,10,1132.291974,21,10,16,1.000000,6,0,0.0,0.0,0.0,0.0,0.00864,6,False,30,2012,10,2,1,2012,10,2,1,2012,10,3,2,3,7,0,862,8.0,9,False,False,False,0.0,0.0,0.0,0,0,0,0.943107,-0.005689,0.030430,0.155310,-0.116201,0.012002,-0.022743,0.000193,0.090681,-0.024496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0.0,0.0,0.0,0.0,0,290,122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1328889185,id,ID,1990,male,1328832000,Binjai,480.0,3.606396,98.489987,1351209600,Medan,,,Indonesia,3.567,98.650,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,37,0.20736,0.035631,212,11.373541,21,10,16,0.041667,154,0,0.0,0.0,0.0,0.0,0.20736,154,False,30,2012,10,2,1,2012,10,2,1,2012,10,26,4,3,5,0,862,8.0,2,False,False,True,0.0,0.0,0.0,0,0,0,0.963601,-0.033379,0.011909,-0.069186,-0.050024,-0.031108,-0.008339,0.001185,0.044168,-0.012568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,2,0,0.0,0.0,0.0,0.0,0,290,3844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1328889185,id,ID,1990,male,1328832000,Binjai,480.0,3.606396,98.489987,1339286400,,,,,34.017,71.583,1,2,1,1,0,0,2,0,0,1,1,1,2,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,2,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3,0,56,0.03456,0.000000,8,2716.054626,21,10,16,0.250000,6,0,0.0,0.0,0.0,0.0,0.03456,6,False,30,2012,10,2,1,2012,10,2,1,2012,10,6,5,0,0,0,862,8.0,8,False,False,True,0.0,0.0,0.0,0,0,0,0.948925,-0.011734,-0.058520,-0.036975,0.003561,-0.031526,-0.027086,-0.003002,0.013024,-0.056438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0.0,0.0,0.0,0.0,0,290,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,1328889185,id,ID,1990,male,1328832000,Binjai,480.0,3.606396,98.489987,1339286400,Medan,,,Indonesia,3.607,98.653,2,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,79,0.03456,0.014417,83,11.253654,21,10,16,0.250000,65,0,0.0,0.0,0.0,0.0,0.03456,65,False,30,2012,10,2,1,2012,10,2,1,2012,10,6,5,3,5,0,862,8.0,6,False,False,True,0.0,0.0,0.0,0,0,0,0.989713,-0.070

In [ ]:
tours_useful_h = pd.read_csv("Data/tours_useful.csv")
bikers_useful_h = pd.read_csv("Data/bikers_useful.csv")
fig, axs = plt.subplots(1, 1, 
                        figsize =(10, 7),  
                        tight_layout = True) 
  
axs.hist(np.array(bikers_useful_h['latitude']), bins = 30) 
  
# Show plot 
plt.show() 

In [ ]:
#tours[tours["tour_id"] == "TP58829330"]
#X_train.shape
latlist = []
longlist = []
for tid, row in tours.items():
    if row[6] != '':
        latlist.append((float(row[6])))
    if row[7] != '':
        longlist.append((float(row[7])))
        
pd.DataFrame(longlist)[0].describe()

In [ ]:
dire = "Data"
biker=pd.read_csv(dire + "/bikers_useful.csv")
tours=pd.read_csv(dire + "/tours_useful.csv")
biker["latitude"] = biker["latitude"].replace(0, 3)
biker["longitude"] = biker["longitude"].replace(0, 100)
tours["latitude"] = tours["latitude"].fillna(3)
tours["longitude"] = tours["longitude"].fillna(100)
biker.to_csv(dire + "/bikers_useful.csv", index = False)
tours.to_csv(dire + "/tours_useful.csv", index = False)
tours.head()

In [ ]:
# locat = geolocator.reverse("41.25007, -69.9963491")
# str(locat).split(',')
for index, row in train.iterrows():
    bid = row['biker_id']
    tse = row['timestamp']
    print(get_lang(bid.upper()), bid.upper())

In [ ]:
top_10_loc = {}
gg.union(top_10_countries)
for coun in gg:
    nam = pycountry.countries.get(alpha_2=coun)
    name = 0.0
    if nam != None:
        name = nam.name
    else:
        continue
    loca = geolocator.geocode(name)
    if loca == None:
        print(name)
        continue
    top_10_loc[coun] = [loca.latitude, loca.longitude]
top_10_loc

In [ ]:
# biker_aba= pd.read_csv("temp/bikers_useful.csv")
# biker_aba.head()
# for index, row in train.iterrows():
#     b1 = row["biker_id"]
#     b2 = tours[row["tour_id"]][0]
#     print(b2 in biker_net)
b = []
for bike1 in tqdm(test_bikers_set):
    idxa = np.where(bike1==test["biker_id"]) 
    tourd = list(test["tour_id"].loc[idxa]) # for each unique biker in test data get all the events  
    for toud in tourd:
        b.append(toud)
b=np.array(b)
pd.DataFrame(b).to_csv("Data/test_tids.csv", header=None)

In [ ]:
tours['SY28440935'][2] == ''
print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

In [ ]:
#pd.DataFrame(test_X).to_csv("Data/test_x.csv", header=None)
nam = pycountry.countries.get(alpha_2='CA')
loca = geolocator.geocode(nam.name)
print(loca.latitude, loca.longitude)